In [3]:
from src.core import Term, Atom
from src.ilp import Language_Frame, Program_Template, Rule_Template
from src.dilp import DILP
import tensorflow as tf
tf.enable_eager_execution()

from src.core import Term, Atom

In [1]:
import pickle

with open("../../uob-summer-project/data/ILP/nodes",'rb') as f:
    nodes = pickle.load(f)
    
with open("../../uob-summer-project/data/ILP/nodes_g",'rb') as f:
    nodes_g = pickle.load(f)

with open("../../uob-summer-project/data/ILP/nodes_b",'rb') as f:
    nodes_b = pickle.load(f)

In [ ]:
# load data

In [16]:
import pandas as pd
prolog = pd.read_csv("../../uob-summer-project/data/ilp_prolog_data.csv.gz")

In [19]:
import re
def fixprolog(testcase):
    find_node_ids = re.compile('\((\w+), (\w+)\)')
    replacement_node_ids = '({bug}_{testcase_id}_\\1, {bug}_{testcase_id}_\\2)'.format(
        bug='bad' if testcase.bug else 'good',
        testcase_id=abs(testcase.testcase_ID),
    )
    return find_node_ids.sub(replacement_node_ids, testcase['tree'])
prolog['tree'] = prolog.apply(fixprolog, axis='columns')
good_example = prolog[prolog['bug'] == False]
bad_example = prolog[prolog['bug'] == True]

def extract_node_ids(tree):
    nodes = set()
    edglist = []
    #astlist = []
    #cfglist = []
    for line in tree.split('\n'):
        line = line.strip()
        if not line.startswith("%") and line:
            parent, child = line[4:-2].split(", ")
            edglist.append([parent, child])
            nodes.add(parent)
            nodes.add(child)
            
    return nodes, edglist

In [45]:
edglist_sum = []
# good nodes_g
nodes_g = set()

for i in range(len(good_example)):
    good_nodes, edglist  = extract_node_ids(good_example.iloc[i].tree)
    nodes_g = set.union(nodes_g,good_nodes)
    edglist_sum.extend(edglist)

# bad nodes_b
nodes_b = set()


for i in range(len(bad_example)):
    bad_nodes, edglist = extract_node_ids(bad_example.iloc[i].tree)
    nodes_b = set.union(nodes_b,bad_nodes)
    edglist_sum.extend(edglist)
        

nodes = set.union(nodes_g,nodes_b)



In [48]:
edglist_sum[1]

['good_62804_good_62804_49', 'good_62804_good_62804_25']

In [49]:
B = [Atom([Term(False, parent), Term(False, child)], 'edglist') for parent, child in edglist_sum]


In [5]:
term_x_0 = Term(True, 'X_0')
term_x_1 = Term(True, 'X_1')

In [51]:
#B = [Atom([Term(False, parent)], [Term(False, child)], 'edglist') for parent,child in astlist]
#B = B.append([Atom([Term(False, parent)], [Term(False, child)], 'cfg') for parent,child in cfg])
P = [Atom([Term(False, nodes)], 'bug') for nodes in nodes_b]
N = [Atom([Term(False, nodes)], 'bug') for nodes in nodes_g]
p_e = [Atom([term_x_0], 'bug'), Atom([term_x_0, term_x_1], 'ast'), Atom([term_x_0, term_x_1], 'cfg')]
p_a = []
constants = [node for node in nodes]
target = Atom([term_x_0], 'bug')
# target_rule = (Rule_Template(0, False), Rule_Template(1, True))
target_rule = (Rule_Template(1, False), None)
rules = {target: target_rule}

In [ ]:
langage_frame = Language_Frame(target, p_e, constants)
program_template = Program_Template(p_a, rules, 10)
program_template
dilp = DILP(langage_frame, B, P, N, program_template)
dilp.train()

In [ ]:
# dilp examples

In [8]:
P = [Atom([Term(False, str(i + 1)), Term(False, str(i))], 'target')
     for i in range(0, 9)]
P

[target(1,0),
 target(2,1),
 target(3,2),
 target(4,3),
 target(5,4),
 target(6,5),
 target(7,6),
 target(8,7),
 target(9,8)]

In [ ]:
N = []
for i in range(0, 10):
    for j in range(0, 10):
        if j != i + 1:
            N.append(
                Atom([Term(False, str(j)), Term(False, str(i))], 'target'))
N

In [ ]:
term_x_0 = Term(True, 'X_0')
term_x_1 = Term(True, 'X_1')
[Atom([term_x_0, term_x_1], 'succ2'), Atom([term_x_0], 'even')]

In [ ]:
term_x_0 = Term(True, 'X_0')
term_x_1 = Term(True, 'X_1')

#p_e = [Atom([term_x_0], 'zero'), Atom([term_x_0], 'even'), Atom([term_x_0, term_x_1], 'succ'), Atom([term_x_0, term_x_1], 'succ2')]
p_a = []
p_e = [Atom([term_x_0], 'zero'), Atom([term_x_0, term_x_1], 'succ'), Atom([term_x_0, term_x_1], 'succ2'), Atom([term_x_0], 'even')]

In [ ]:
p_e = [Atom([term_x_0], 'zero'), Atom([term_x_0, term_x_1], 'succ')]
p_a = []

target = Atom([term_x_0, term_x_1], 'target')
# target_rule = (Rule_Template(0, False), Rule_Template(1, True))
target_rule = (Rule_Template(1, False), None)
rules = {target: target_rule}
constants = [str(i) for i in range(0, 10)]

In [1]:
langage_frame = Language_Frame(target, p_e, constants)
program_template = Program_Template(p_a, rules, 10)
program_template
dilp = DILP(langage_frame, B, P, N, program_template)
dilp.train()

Performing Inference
Instructions for updating:
Colocations handled automatically by placer.
Progress: |██████████████████████████████████████████████████| 100.0% Complete
Inference Complete
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Performing Inference
Progress: |██████████████████████████████████████████████████| 100.0% Complete
Inference Complete
--------------------
step 0 loss is 0.5105849504470825
Performing Inference
Progress: |██████████████████████████████████████████████████| 100.0% Complete
Inference Complete
target(0,0) Expected: 0 0.871
target(0,1) Expected: 0 0.948
target(0,2) Expected: 0 0.631
target(0,3) Expected: 0 0.598
target(0,4) Expected: 0 0.598
target(0,5) Expected: 0 0.598
target(0,6) Expected: 0 0.598
target(0,7) Expected: 0 0.598
target(0,8) Expected: 0 0.598
target(0,9) Expected: 0 0.554
target(1,0) Expected: 1 0.907
target(1,1) Expected: 0 0.550
target(1,2) Expected: 0 0.855


Performing Inference
Progress: |██████████████████████████████████████████████████| 100.0% Complete
Inference Complete
--------------------
step 6 loss is 0.5055342316627502
--------------------

Performing Inference


KeyboardInterrupt: 